In [5]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os

from torch.utils.data import DataLoader
from utils.dataset import EchoDataset
import matplotlib.pyplot as plt

import cv2
from tqdm import tqdm

In [7]:
dataset_dir = '../dataset/EchoNet-LVH'
# dataset_dir = 'dataset'

In [8]:
ds = EchoDataset(root=dataset_dir)

In [9]:
dl = DataLoader(ds, batch_size=1, shuffle=False)

In [10]:
def write_image(image, nth, directory, ext='png'):
    filename = f"{nth:04d}.{ext}"
    os.makedirs(directory, exist_ok=True)
    try:
        cv2.imwrite(os.path.join(directory, filename), image)
    except Exception as e:
        print(f"{e} - {filename, nth} Error")

In [8]:
for nth, sample in tqdm(enumerate(dl), total=dl.__len__()):
    video = sample['video'].squeeze().permute(1,0,2,3)
    directory = sample['id'][0]
    for i, frame in tqdm(enumerate(video, start=1), total=len(video), leave=True):
        frame = frame.permute(1,2,0).numpy()
        write_image(frame, i, os.path.join('output', directory.split('.')[0]))

100%|██████████| 11511/11511 [5:54:10<00:00,  1.85s/it]


# Filtering 및 Save

레이블링이 존재하는 프레임만 추출

In [14]:
import pandas as pd
data = pd.read_csv(os.path.join(dataset_dir, 'MeasurementsList.csv'), index_col=0)
data['HashedFileName'] = data['HashedFileName'].map(lambda x: x+'.avi')

# 3개의 distole 좌표값이 있는 경우에만 살림
calc_list = ['IVSd', 'LVIDd', 'LVPWd']#, 'IVSs', 'LVIDs', 'LVPWs']
data = data[data['Calc'].apply(lambda x: x in calc_list)].reset_index(drop=True)
perfect_list = [filename for filename in data['HashedFileName'].unique() if len(data[data['HashedFileName'] == filename])==3]
data = data[data['HashedFileName'].apply(lambda x: x in perfect_list)].reset_index(drop=True)


In [15]:
# # 필터링
# new_df = data[['HashedFileName', 'Frame']].drop_duplicates()
# for i in [' COLMAN', ' GENEVIE', 'JR^ROBERTO^E^^']:
#     tmp = new_df['HashedFileName'][new_df['Frame'] ==  i]
#     new_df = new_df[new_df['HashedFileName'] != tmp.values[0]]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [20]:
import shutil
output_dir2 = 'output2'
os.makedirs(output_dir2, exist_ok=True)
for i, j in zip(new_df['HashedFileName'].apply(lambda x: x.split('.')[0]), new_df['Frame'].apply(lambda x: f"{int(x):04d}")):
    source_path = os.path.join('output', i, str(j)+".png")
    dest_path = os.path.join(output_dir2, f"{i}_{str(j)}.png")

    shutil.copy(source_path, dest_path)